## Load sample questions

In [ ]:
import json

# Load the sample ENEM questions
with open("enem_questions_sample.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Preview the first question
data[0]

## Connect to OpenAI API

In [26]:
from openai import OpenAI

# Read key from file
with open("openai-key.txt", "r") as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key)

### Define API call

In [27]:
def call_openai_api(prompt, model="gpt-3.5-turbo", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

### Define answer extractor

In [69]:
import re

def extract_answer_letter(response):
    # Look for something like: "Resposta final: D) ..." or "Resposta final: D"
    match = re.search(r"Resposta final\s*[:\-]?\s*([A-E])\s?\)", response, re.IGNORECASE)
    if not match:
        match = re.search(r"Resposta final\s*[:\-]?\s*([A-E])\b", response, re.IGNORECASE)
    return match.group(1) if match else None

## Define a Chain-of-Thought (CoT) template

In [75]:
def build_cot_prompt(question_obj):
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{letter}) {text}" for letter, text in zip(option_letters, options)])
    prompt = f"""Responda à seguinte pergunta com raciocínio passo a passo, explicando sua escolha antes de dar a resposta final.

Pergunta:
{question}

Opções:
{formatted_options}

Explique sua resposta e depois diga a letra da alternativa correta no formato "Resposta final: X"
"""
    return prompt


### Mock CoT model response for testing

In [76]:
import random

def mock_model_response(prompt, technique="cot"):
    # Randomly pick an answer letter from A to E
    answer_letter = random.choice(["A", "B", "C", "D", "E"])

    # Simulated explanation for Chain-of-Thought
    explanation = f"""Claro! Vamos pensar passo a passo...

[Simulação de raciocínio baseado no conteúdo da pergunta]

Resposta final: {answer_letter}"""
    
    return explanation

In [77]:
import re

def extract_answer_letter(response):
    match = re.search(r"Resposta final: ([A-E])", response)
    if match:
        return match.group(1)
    return None


### Test CoT template with a mock response

In [78]:
q = data[0]
prompt = build_cot_prompt(q)
response = mock_model_response(prompt)
predicted = extract_answer_letter(response)

print("Prompt:")
print(prompt)
print("\nMocked Model Response:")
print(response)
print(f"\nPredicted: {predicted} | Ground Truth: {q['answer']}")


Prompt:
Responda à seguinte pergunta com raciocínio passo a passo, explicando sua escolha antes de dar a resposta final.

Pergunta:
Impactos do microplástico
A ação de fatores abióticos aliada à biodeterioração contribuem para a formação de microplásticos, os quais se aderem a outros poluentes orgânicos apolares persistentes, como os derivados de pesticidas lipossolúveis. Há uma proporcionalidade direta entre a solubilidade desses tipos de poluentes e sua concentração nos tecidos dos organismos expostos a eles.
Em animais vertebrados, essa associação de poluentes será preferencialmente acumulada no tecido

Opções:
A) ósseo.
B) nervoso.
C) epitelial.
D) adiposo.
E) sanguíneo.

Explique sua resposta e depois diga a letra da alternativa correta no formato "Resposta final: X"


Mocked Model Response:
Claro! Vamos pensar passo a passo...

[Simulação de raciocínio baseado no conteúdo da pergunta]

Resposta final: B

Predicted: B | Ground Truth: D


## Define a Chain-of-Verification (CoVe) template

### Plan verifications

In [28]:
def plan_verification_questions(question, baseline_answer):
    plan_prompt = f"""Dada a seguinte pergunta e resposta, gere 2 a 4 perguntas para verificar os fatos principais da resposta.

Pergunta: {question}

Resposta: {baseline_answer}

Liste as perguntas de verificação:"""
    return call_openai_api(plan_prompt)


### Execute verifications

In [29]:
def execute_verifications(verification_questions):
    verifications = []
    for q in verification_questions:
        answer = call_openai_api(q)
        verifications.append((q, answer))
    return verifications

### Generate final verified answer

In [79]:
def generate_final_verified_answer(question, original_answer, verifications, options=None):
    vtext = "\n".join([f"Q: {q}\nA: {a}" for q, a in verifications])

    option_letters = ["A", "B", "C", "D", "E"]
    options_text = ""
    if options:
        options_text = "\n".join([f"{letter}) {text}" for letter, text in zip(option_letters, options)])

    revise_prompt = f"""
Revise a resposta abaixo com base nas verificações.

Pergunta:
{question}

Alternativas:
{options_text}

Resposta original:
{original_answer}

Verificações:
{vtext}

Resposta final verificada:
[Inclua uma explicação revisada, seguida de uma linha como: "Resposta final: X", onde X é a letra da alternativa correta.]
"""
    return call_openai_api(revise_prompt)


## Define a Self-Refine template

### Ask for feedback

In [85]:
def build_feedback_prompt(question_obj, model_output):
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{l}) {t}" for l, t in zip(option_letters, options)])

    prompt = (
        "Analise a seguinte resposta gerada para uma pergunta do ENEM. "
        "Identifique erros, falhas na argumentação ou escolha incorreta da alternativa. "
        "Aponte aspectos que podem ser melhorados.\n\n"
        f"Pergunta:\n{question}\n\n"
        f"Alternativas:\n{formatted_options}\n\n"
        f"Resposta do modelo:\n{model_output}\n\n"
        "Feedback:"
    )
    return prompt


### Ask for refined response based on feedback

In [86]:
def build_refine_prompt(question_obj, model_output, feedback):
    question = question_obj["question"]
    options = question_obj["alternatives"]
    option_letters = ["A", "B", "C", "D", "E"]
    formatted_options = "\n".join([f"{l}) {t}" for l, t in zip(option_letters, options)])

    prompt = (
        "A seguir está uma pergunta do ENEM, acompanhada de alternativas, "
        "uma resposta inicial e um feedback crítico. Escreva uma nova resposta levando em conta "
        "o feedback, explicando novamente o raciocínio e indicando a letra da alternativa correta "
        "no formato \"Resposta final: X\".\n\n"
        f"Pergunta:\n{question}\n\n"
        f"Alternativas:\n{formatted_options}\n\n"
        f"Resposta anterior:\n{model_output}\n\n"
        f"Feedback:\n{feedback}\n\n"
        "Nova resposta:"
    )
    return prompt


### Iteration Wrapper

In [87]:
def self_refine_enem(question_obj, max_iters=3):
    prompt = build_cot_prompt(question_obj)
    response = call_openai_api(prompt)
    
    history = [(response, None)]

    for _ in range(max_iters):
        fb_prompt = build_feedback_prompt(question_obj, response)
        feedback = call_openai_api(fb_prompt)

        if "nenhuma alteração" in feedback.lower() or "satisfatória" in feedback.lower():
            break

        refine_prompt = build_refine_prompt(question_obj, response, feedback)
        response = call_openai_api(refine_prompt)
        history.append((response, feedback))
    
    return response, history


## Tests

### Helpers

In [54]:
import pandas as pd

def calculate_accuracy(results):
    """Returns overall accuracy from a list of result dicts."""
    correct = sum(r["correct"] for r in results if r["predicted"] is not None)
    total = sum(1 for r in results if r["predicted"] is not None)
    accuracy = (correct / total) * 100 if total > 0 else 0
    return correct, total, accuracy


def results_by_subject(results):
    """Aggregates accuracy grouped by subject (assumes 'subject' is in each result)."""
    df = pd.DataFrame(results)
    if "subject" not in df.columns:
        print("⚠️ 'subject' not found in results.")
        return None
    
    summary = df.groupby("subject")["correct"].agg(["sum", "count"])
    summary["accuracy (%)"] = (summary["sum"] / summary["count"]) * 100
    return summary

### Test CoT with GPT

#### Test CoT with a single question

In [15]:
q = data[0]
prompt = build_cot_prompt(q)
response = call_openai_api(prompt)
predicted = extract_answer_letter(response)

print("Prompt:\n", prompt)
print("\nModel Response:\n", response)
print(f"\nPredicted: {predicted} | Ground Truth: {q['answer']}")


Prompt:
 Responda à seguinte pergunta com raciocínio passo a passo, explicando sua escolha antes de dar a resposta final.

Pergunta:
Impactos do microplástico
A ação de fatores abióticos aliada à biodeterioração contribuem para a formação de microplásticos, os quais se aderem a outros poluentes orgânicos apolares persistentes, como os derivados de pesticidas lipossolúveis. Há uma proporcionalidade direta entre a solubilidade desses tipos de poluentes e sua concentração nos tecidos dos organismos expostos a eles.
Em animais vertebrados, essa associação de poluentes será preferencialmente acumulada no tecido

Opções:
A) ósseo.
B) nervoso.
C) epitelial.
D) adiposo.
E) sanguíneo.

Explique sua resposta e depois diga a letra da alternativa correta no formato "Resposta final: X"


Model Response:
 Para responder a essa pergunta, devemos considerar que os microplásticos e poluentes orgânicos apolares persistentes são substâncias que podem se acumular nos tecidos dos organismos expostos a eles

#### CoT evaluation loop over sample questions

In [42]:
cot_results = []

for q in data:
    prompt = build_cot_prompt(q)
    try:
        response = call_openai_api(prompt)
        predicted = extract_answer_letter(response)
    except Exception as e:
        response = str(e)
        predicted = None
    
    cot_results.append({
        "id": q["id"],
        "subject": q["subject"],
        "ground_truth": q["answer"],
        "predicted": predicted,
        "correct": predicted == q["answer"],
        "response": response
    })


#### CoT Results

In [55]:
df_cot = pd.DataFrame(cot_results)
df_cot

,id,subject,ground_truth,predicted,correct,response
0,questao_94,Biologia,D,D,True,"Para responder a essa pergunta, devemos analisar o tipo de poluentes orgânicos apolares persistentes que se aderem aos microplásticos, que são os derivados de pesticidas lipossolúveis. Sabemos que compostos lipossolúveis têm afinidade por tecidos adiposos, devido à sua natureza lipofílica.\n\nPortanto, levando em consideração que a associação de poluentes será preferencialmente acumulada nos tecidos que possuem maior afinidade por compostos lipossolúveis, a resposta correta é:\n\nResposta final: D) adiposo."
1,questao_95,Física,D,D,True,"Para justificar o procedimento de retirar os computadores portáteis das malas, mochilas ou bolsas e passá-los isoladamente pela esteira de raios X, é importante considerar a interação dos raios X com os componentes do computador. Os raios X são capazes de atravessar materiais menos densos, como tecidos e plásticos, mas podem ser bloqueados por materiais mais densos, como metais.\n\nDessa forma, ao passar o computador isoladamente pela esteira de raios X, é possível garantir que os agentes de segurança possam visualizar com clareza todos os componentes do dispositivo, sem a interferência de outros materiais presentes na bagagem de mão que possam dificultar a inspeção.\n\nResposta final: D) Os raios X não atravessam os componentes densos do computador, o que impede a visualização de objetos que estão à frente ou atrás deles."
2,questao_96,Química,E,E,True,"O processo de formação de compostos nitrosos a partir do nitrito de sódio e ácido clorídrico ocorre no estômago, uma vez que é no estômago que ocorre a digestão dos alimentos, incluindo a digestão de proteínas. Durante a digestão de alimentos ricos em proteínas, como carnes, as aminas presentes nesses alimentos podem reagir com o ácido nitroso produzido a partir do nitrito de sódio, resultando na formação de compostos nitrosos mutagênicos.\n\nResposta final: E) Estômago"
3,questao_97,Biologia,B,B,True,"Para responder a essa pergunta, é importante entender o conceito de sistemas agroflorestais (SAFs) e como eles se relacionam com processos naturais. Os SAFs são sistemas de produção agrícola que combinam árvores, culturas agrícolas e/ou animais no mesmo espaço, de forma integrada e sustentável. Essa integração beneficia tanto a produção econômica quanto a conservação ambiental.\n\nAo analisar as opções dadas, podemos descartar algumas delas. Por exemplo, a rotação de culturas é um sistema de manejo agrícola que envolve alternar diferentes culturas em um mesmo espaço ao longo do tempo, mas não necessariamente se relaciona com a integração de árvores e animais, característica dos SAFs.\n\nA adaptação por seleção e a convergência adaptativa são processos evolutivos que ocorrem ao longo de gerações em populações de organismos, não sendo aplicáveis ao contexto dos sistemas agroflorestais.\n\nA coevolução específica é um processo em que duas espécies interagem e evoluem em resposta uma à outra, o que não abrange necessariamente a integração de elementos vegetais e animais em um mesmo sistema de produção.\n\nA sucessão ecológica, por sua vez, é um processo natural em que a composição e estrutura de uma comunidade biológica mudam ao longo do tempo, em um padrão previsível. Esse processo envolve a substituição gradual de espécies em um determinado ambiente, levando à formação de comunidades mais complexas e estáveis.\n\nDessa forma, a sucessão ecológica é o processo natural análogo às mudanças graduais e integradas observadas nos sistemas agroflorestais, já que ambos envolvem a transição para estados mais complexos e estáveis ao longo do tempo.\n\nResposta final: B) Sucessão ecológica."
4,questao_98,Biologia,B,B,True,"Para responder a essa pergunta, é importante entender a diferença entre as fibras musculares brancas e vermelhas. As fibras musculares brancas são mais rápidas, mas se fadigam mais rapidamente, enquanto as fibras musculares vermelhas são mais resistentes à f

#### Overall accuracy

In [57]:
correct, total, acc = calculate_accuracy(cot_results)
print(f"CoVe Accuracy: {correct}/{total} ({acc:.1f}%)")

CoVe Accuracy: 6/6 (100.0%)


#### Accuracy by Subject

In [58]:
results_by_subject(pd.DataFrame(cot_results))

,sum,count,accuracy (%)
subject,,,
Biologia,3,3,100.0
Física,2,2,100.0
Química,1,1,100.0


### Test CoVe with GPT

#### Test CoVe with a single question

In [70]:
# Step 1: Pick a question
q = data[0]
question_text = q["question"]
prompt = build_cot_prompt(q)

# Step 2: Baseline CoT answer
baseline_answer = call_openai_api(prompt)

print("🔹 Baseline CoT Answer:\n", baseline_answer)

# Step 3: Plan verification questions
verification_qs_raw = plan_verification_questions(question_text, baseline_answer)

# Split the response into individual questions if the model gives a list
verification_questions = [line.strip("- ").strip() for line in verification_qs_raw.split("\n") if line.strip()]

print("\n🔹 Verification Questions:")
for qv in verification_questions:
    print("-", qv)

# Step 4: Execute verification
verifications = execute_verifications(verification_questions)

print("\n🔹 Verification Answers:")
for qv, av in verifications:
    print(f"Q: {qv}\nA: {av}\n")

# Step 5: Generate final verified answer
final_answer = generate_final_verified_answer(question_text, baseline_answer, verifications)

print("🔹 Final Verified Answer:\n", final_answer)

# Step 6: Extract predicted letter and compare to ground truth
predicted = extract_answer_letter(final_answer)
ground_truth = q["answer"]

print(f"\n✅ Predicted Answer: {predicted}")
print(f"🎯 Ground Truth: {ground_truth}")

if predicted == ground_truth:
    print("🎉 CORRECT!")
else:
    print("❌ WRONG.")


🔹 Baseline CoT Answer:
 Para responder a essa pergunta, devemos analisar a associação de poluentes apolares persistentes, como os microplásticos, com outros poluentes orgânicos lipossolúveis, e como isso afeta os tecidos dos animais vertebrados expostos a eles.

Os poluentes lipossolúveis têm maior afinidade por tecidos adiposos, devido à sua natureza lipofílica. Portanto, os poluentes associados aos microplásticos serão preferencialmente acumulados nos tecidos adiposos dos animais vertebrados.

Resposta final: D) adiposo.

🔹 Verification Questions:
- 1. Como a ação de fatores abióticos contribui para a formação de microplásticos?
- 2. Qual a relação entre os microplásticos e os poluentes orgânicos apolares persistentes?
- 3. Por que os poluentes associados aos microplásticos são preferencialmente acumulados nos tecidos adiposos dos animais vertebrados?
- 4. Qual a importância da solubilidade dos poluentes na concentração nos tecidos dos organismos expostos a eles?

🔹 Verification Answ

#### CoVe evaluation loop over sample questions

In [71]:
cove_results = []

for i, q in enumerate(data):
    try:
        question_text = q["question"]
        correct_answer = q["answer"]

        # 1. Build CoT-style prompt
        prompt = build_cot_prompt(q)

        # 2. Baseline CoT response
        baseline_answer = call_openai_api(prompt)
        
        # 3. Plan verifications
        verification_qs_raw = plan_verification_questions(question_text, baseline_answer)
        verification_questions = [line.strip("- ").strip() for line in verification_qs_raw.split("\n") if line.strip()]

        # 4. Execute verifications
        verifications = execute_verifications(verification_questions)

        # 5. Final revised answer
        final_answer = generate_final_verified_answer(
            question_text, baseline_answer, verifications, q["alternatives"]
        )

        # 6. Extract answer letter (A–E)
        predicted = extract_answer_letter(final_answer)

        # 7. Record result
        cove_results.append({
            "index": i,
            "question": question_text,
            "true_answer": correct_answer,
            "predicted": predicted,
            "correct": predicted == correct_answer,
            "baseline_answer": baseline_answer,
            "final_answer": final_answer,
            "verification_qs": verification_questions,
            "verification_a": verifications,
            "subject": q.get("subject", "unknown")
        })

        print(f"[{i+1}/{len(data)}] ✅ Predicted: {predicted} | Correct: {correct_answer}")

    except Exception as e:
        print(f"[{i+1}/{len(data)}] ❌ Error: {e}")
        cove_results.append({
            "index": i,
            "question": question_text,
            "true_answer": correct_answer,
            "predicted": None,
            "correct": False,
            "error": str(e)
        })


[1/6] ✅ Predicted: D | Correct: D
[2/6] ✅ Predicted: D | Correct: D
[3/6] ✅ Predicted: E | Correct: E
[4/6] ✅ Predicted: B | Correct: B
[5/6] ✅ Predicted: B | Correct: B
[6/6] ✅ Predicted: A | Correct: A


#### CoVe results

In [72]:
df_cove = pd.DataFrame(cove_results)
df_cove

index  \
0      0   
1      1   
2      2   
3      3   
4      4   
5      5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             question  \
0  Impactos do microplástico\nA ação de fatores abióticos aliada à biodeterioração contribuem para a formação de microplásticos, os quais se aderem a outros poluentes orgânicos apolares persistentes, como os derivados de pesticidas lipossolúveis. Há uma proporcionalidade direta entre a solubilidade desses tipos de poluentes e sua concentração nos tecidos dos organismos expostos a eles.\nEm animais vertebrados, essa associação de poluentes será preferencialmente acumulada no tecido   
1                                                                     Em aeroportos, por razões de segurança, os passageiros devem ter suas bagagens de mão examinadas antes do embarque, passando-as em esteiras para sua inspeção por aparelhos de raios X. Nessas inspeções, os passageiros são orientados a retirar seus computadores portáteis (notebooks ou laptops) de malas, mochilas ou bolsas para passá-los isoladamente pela esteira.\nQue explicação física justifica esse procedimento?   
2                Com base em testes realizados em ratos, concluiu-se que os compostos nitrosos são substâncias mutagênicas, ou seja, produzem mutações nas células e possivelmente câncer. Esses compostos podem ser obtidos pela reação entre o nitrito de sódio, que é um conservante adicionado às carnes, e o ácido clorídrico. O ácido nitroso produzido irá reagir com compostos nitrogenados, como as aminas, dando origem aos compostos nitrosos.\nEm qual órgão esse processo será iniciado?   
3                                                                                                                                                                                                                                                                                                      Sistemas agroflorestais (SAFs)\nOs sistemas agroflorestais alinham os interesses econômicos aos ecológicos. (...) O conjunto dessas mudanças graduais é análogo ao processo natural denominado   
4                                                                                                                                                                                                                                      Vida: a ciência da biologia\nAs fibras musculares esqueléticas não são todas iguais. (...) Um atleta que sonha em disputar os Jogos Olímpicos e tem uma maior proporção de fibras brancas que fibras vermelhas teria mais vantagens na realização da prova de:   
5                                                                                                                                                                                                                                                                                A saúde do professor: acústica arquitetônica\n(...) Para um valor máximo do ruído de fundo, a maior distância que um estudante pode estar do professor para que ainda consiga compreender sua fala é mais próxima de   

  true_answer predicted  correct  \
0           D         D     True   
1           D         D     True   
2           E         E     True   
3           B         B     True   
4           B         B     True   
5           A         A     True   

                                                                                                                                                                                                                                                 

#### Overall accuracy

In [73]:
correct, total, acc = calculate_accuracy(cove_results)
print(f"CoVe Accuracy: {correct}/{total} ({acc:.1f}%)")

CoVe Accuracy: 6/6 (100.0%)


#### Accuracy by Subject

In [74]:
results_by_subject(pd.DataFrame(cove_results))

,sum,count,accuracy (%)
subject,,,
Biologia,3,3,100.0
Física,2,2,100.0
Química,1,1,100.0


### Test Self-Refine

#### Test Self-Refine with a single question 

In [88]:
q = data[0]
final_response, trace = self_refine_enem(q)
predicted = extract_answer_letter(final_response)

print("Prompt inicial (iteração 0):\n", build_cot_prompt(q))
print("\nResposta final após refinamento:\n", final_response)
print(f"\nAlternativa prevista: {predicted} | Gabarito: {q['answer']}")

print("\nHistórico de iterações:")
for i, (resp, fb) in enumerate(trace):
    print(f"\n--- Iteração {i} ---")
    print("Resposta:", resp)
    if fb:
        print("Feedback:", fb)


Prompt inicial (iteração 0):
 Responda à seguinte pergunta com raciocínio passo a passo, explicando sua escolha antes de dar a resposta final.

Pergunta:
Impactos do microplástico
A ação de fatores abióticos aliada à biodeterioração contribuem para a formação de microplásticos, os quais se aderem a outros poluentes orgânicos apolares persistentes, como os derivados de pesticidas lipossolúveis. Há uma proporcionalidade direta entre a solubilidade desses tipos de poluentes e sua concentração nos tecidos dos organismos expostos a eles.
Em animais vertebrados, essa associação de poluentes será preferencialmente acumulada no tecido

Opções:
A) ósseo.
B) nervoso.
C) epitelial.
D) adiposo.
E) sanguíneo.

Explique sua resposta e depois diga a letra da alternativa correta no formato "Resposta final: X"


Resposta final após refinamento:
 Os microplásticos, juntamente com outros poluentes orgânicos apolares persistentes, têm a capacidade de se acumular nos tecidos dos animais vertebrados. Devido

#### Self-Refine evaluation loop over sample questions

In [89]:
self_refine_results = []

for i, q in enumerate(data):
    try:
        question_text = q["question"]
        correct_answer = q["answer"]

        # 1. Run SELF-REFINE
        final_answer, trace = self_refine_enem(q, max_iters=3)
        predicted = extract_answer_letter(final_answer)

        # 2. Get initial CoT answer for logging
        baseline_answer = trace[0][0] if trace else None

        # 3. Record result
        self_refine_results.append({
            "index": i,
            "question": question_text,
            "true_answer": correct_answer,
            "predicted": predicted,
            "correct": predicted == correct_answer,
            "baseline_answer": baseline_answer,
            "final_answer": final_answer,
            "trace": trace,  # optional: full iteration history
            "subject": q.get("subject", "unknown")
        })

        print(f"[{i+1}/{len(data)}] ✅ Predicted: {predicted} | Correct: {correct_answer}")

    except Exception as e:
        print(f"[{i+1}/{len(data)}] ❌ Error: {e}")
        self_refine_results.append({
            "index": i,
            "question": q.get("question", ""),
            "true_answer": q.get("answer", ""),
            "predicted": None,
            "correct": False,
            "error": str(e),
            "subject": q.get("subject", "unknown")
        })

[1/6] ✅ Predicted: D | Correct: D
[2/6] ✅ Predicted: D | Correct: D
[3/6] ✅ Predicted: E | Correct: E
[4/6] ✅ Predicted: B | Correct: B
[5/6] ✅ Predicted: E | Correct: B
[6/6] ✅ Predicted: B | Correct: A


#### Self-Refine Results

In [94]:
df_self_refine = pd.DataFrame(self_refine_results)
df_self_refine


index  \
0      0   
1      1   
2      2   
3      3   
4      4   
5      5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             question  \
0  Impactos do microplástico\nA ação de fatores abióticos aliada à biodeterioração contribuem para a formação de microplásticos, os quais se aderem a outros poluentes orgânicos apolares persistentes, como os derivados de pesticidas lipossolúveis. Há uma proporcionalidade direta entre a solubilidade desses tipos de poluentes e sua concentração nos tecidos dos organismos expostos a eles.\nEm animais vertebrados, essa associação de poluentes será preferencialmente acumulada no tecido   
1                                                                     Em aeroportos, por razões de segurança, os passageiros devem ter suas bagagens de mão examinadas antes do embarque, passando-as em esteiras para sua inspeção por aparelhos de raios X. Nessas inspeções, os passageiros são orientados a retirar seus computadores portáteis (notebooks ou laptops) de malas, mochilas ou bolsas para passá-los isoladamente pela esteira.\nQue explicação física justifica esse procedimento?   
2                Com base em testes realizados em ratos, concluiu-se que os compostos nitrosos são substâncias mutagênicas, ou seja, produzem mutações nas células e possivelmente câncer. Esses compostos podem ser obtidos pela reação entre o nitrito de sódio, que é um conservante adicionado às carnes, e o ácido clorídrico. O ácido nitroso produzido irá reagir com compostos nitrogenados, como as aminas, dando origem aos compostos nitrosos.\nEm qual órgão esse processo será iniciado?   
3                                                                                                                                                                                                                                                                                                      Sistemas agroflorestais (SAFs)\nOs sistemas agroflorestais alinham os interesses econômicos aos ecológicos. (...) O conjunto dessas mudanças graduais é análogo ao processo natural denominado   
4                                                                                                                                                                                                                                      Vida: a ciência da biologia\nAs fibras musculares esqueléticas não são todas iguais. (...) Um atleta que sonha em disputar os Jogos Olímpicos e tem uma maior proporção de fibras brancas que fibras vermelhas teria mais vantagens na realização da prova de:   
5                                                                                                                                                                                                                                                                                A saúde do professor: acústica arquitetônica\n(...) Para um valor máximo do ruído de fundo, a maior distância que um estudante pode estar do professor para que ainda consiga compreender sua fala é mais próxima de   

  true_answer predicted  correct  \
0           D         D     True   
1           D         D     True   
2           E         E     True   
3           B         B     True   
4           B         E    False   
5           A         B    False   

                                                                                                                                                                                                                                                 

In [95]:
# Loop through incorrect predictions and show their trace
for r in self_refine_results:
    if not r["correct"]:
        print("="*80)
        print(f"🔎 Question {r['index']+1} | True: {r['true_answer']} | Predicted: {r['predicted']}")
        print("-" * 80)
        for i, (resp, fb) in enumerate(r.get("trace", [])):
            print(f"📌 Iteration {i}")
            print("Resposta:\n", resp.strip())
            if fb:
                print("\n📝 Feedback:\n", fb.strip())
            print("-" * 80)


🔎 Question 5 | True: B | Predicted: E
--------------------------------------------------------------------------------
📌 Iteration 0
Resposta:
 Para responder a essa pergunta, é importante entender a diferença entre as fibras musculares brancas e vermelhas. As fibras brancas são fibras de contração rápida e são mais utilizadas em atividades de força e explosão, enquanto as fibras vermelhas são fibras de contração lenta e são mais utilizadas em atividades de resistência.

Considerando que um atleta com uma maior proporção de fibras brancas teria uma vantagem na realização de atividades que exigem explosão e força, podemos excluir as opções que envolvem resistência e endurance, como o triatlo, a maratona aquática e o ciclismo em estrada.

Portanto, a melhor opção seria uma prova que requer explosão e força, como o salto em altura, que exige um rápido impulso e explosão muscular para alcançar alturas elevadas.

Resposta final: B) Salto em altura.
------------------------------------------

#### Overall accuracy

In [91]:
correct, total, accuracy = calculate_accuracy(self_refine_results)
print(f"✅ SELF-REFINE Accuracy: {accuracy:.2f}% ({correct}/{total})")

✅ SELF-REFINE Accuracy: 66.67% (4/6)


#### Accuracy by Subject

In [92]:
subject_summary = results_by_subject(self_refine_results)
print("\n📊 Accuracy by subject:")
display(subject_summary)


📊 Accuracy by subject:


,sum,count,accuracy (%)
subject,,,
Biologia,2,3,66.666667
Física,1,2,50.000000
Química,1,1,100.000000
